In [32]:
# libraries
import numpy as np
import pandas as pd
import altair as alt
from sklearn.decomposition import PCA

# warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# display settings
pd.options.display.max_columns = None

# PSTAT 100 Project plan

This is a guide to preparing your project plan. It functions both as a guide to the work you'll need to do and as a guide to preparing the deliverable. You can use it as a template to draft the plan report; if so, **please remove the text explanations and instructions in each section so that it reads as a coherent and continuous document**.

While you may find it useful initially to follow the outline given, you do not need to adhere to it exactly -- you're free to organize your submission in the way that seems most natural to you. However, please do keep the high-level sections, so that your report includes the following headers:

0. Background
1. Data description
2. Initial exporations
3. Planned work

Your report does not need to be long. It should be about 2-4 pages, and might not be much longer than this template once you replace the guiding text with your own work.

## Group information

**Group members**:

**Contributions**:
1. Member 1 studied the data documentation and prepared the data description.
2. Member 2 worked on tidying the dataset.
3. Member 3 ...

---
## 0. Background

This section should introduce your reader to the general topic you're engaging with in your project and explain any specialized knowledge that they may need to understand your dataset and why it's interesting. It doesn't need to be long, but should touch on the following points:
* Introduce the topic of your project.
* What area or areas of study are you in dialogue with for your project?
* What is your data about, broadly? 
* What is the motivation for collecting the kind of data you're working with, and what sorts of things could you potentially learn?

You can look to the background sections in the homework assignments for examples. (There you can also see how to include images in your notebook.) The background sections of the homeworks are usually short and focused paragraphs intended to orient you to what you'll do in the assignment. They don't go into a lot of detail -- just enough to (hopefully) convince you that the data are interesting and explain any terminology or general information you may not know.

You may find it useful to write up the data description first, think about what the reader should know before they peek at your dataset, and then come back to the background section. I often write the background sections of your assignments last, once I have a sense of what kind of information would be most useful going into the assignment.

*Start your draft here.*

---
## 1. Data description

### Basic Information

The data is the percent of the population that resides within 1/2 mile of a major transit location (bus/rail/ferry stop) in four California regions, and whose waiting time is less than 15 minutes during peak commute hours. The data is stratified by 8 race/ethnicity groups and includes both geographic information and statistical reliability measurements.

The data was collected by the California Department of Public Health, as part of the "Health Communities Data and Indicators Project (HCI). The goal of the project was to evaluate how city plans and policies affect community health. The data includes 2012 Transit Stops from the San Diego and Southern California Association of Governments, as well as the Metropolitan Transportation Commission. It also includes 2008 Transit Stops from the Sacramento Council of Government and 2010 block-level population data from the U.S. Census Bureau. The data is updated decennially. The four California regions are defined as the following:
* Southern California (SCAG): Imperial, Los Angeles, Orange, Riverside, San Bernardino, and Ventura 
* Sacramento (SACOG): Placer, Sacramento, and Yolo 
* Bay Area (MTC): Contra Costa, Marin, Napa, San Francisco, San Mateo, Santa Clara, Solano, Sonoma 
* San Diego County

Data values were obtained using automated methods to download information from various public websites. One important data set was from the 2010-2012 California Household Travel Survey. Multiple data collection methods were used in this survey, including computer-assisted telephone interviews and online/mail surveys. To identify census blocks inside 1/2 mile of the transit stops, geospatial software was used. In order to compile the data into one data set, the census blocks from the 2010 U.S. Census were merged with the blocks from the transit data, and population counts were aggregated by census tract, cities/towns, county, and region. The data was processed into Excel files with standard formats. 

The population is adults aged 18 years and over, who reside in the four California regions. The sampling frame includes adults in these four regions, with access to telephone or mail services. The sampling mechanism for the respective year (2008 or 2012) is a probability sample because the surveys downloaded by the HCI project were sent to randomly selected adults However, the scope of inference has limitations. The data is from the year 2012 for the SCAG, MTC, and San Diego regions; while, 2008 for the SACOG region. Some transit stops and services may have changed during that time period. As well, the population data was collected from the 2010 U.S. Census, which is a different time period than the transit data (2008, 2012). Therefore, some variation may exist if demographics changed.

### Data semantics and structure

Name | Variable description | Type | Units of measurement
---|---|---|---
year | year when data was reported | Numeric | Calendar year 
race_eth_name | name of the different races/ethnicties ('AfricanAm', 'AIAN', 'Asian', 'Latino', 'Multiple', 'NHOPI', 'Other', 'Total', 'White')| Object | Name
geotype | describes the level of geography for data in that row ('RE'=region, 'CT'=census tract, 'PL'=place/town/city, 'CO'=county)  | Object | Name
geoname | name of the city/town | Object | Name 
county_name | name of the county | Object | Name
region_name | name of the region ('Sacramento Area', 'Bay Area', 'San Diego', 'Southern California') | Object | Name
pop_trans_acc | number of residents that live within 1/2 mile of public transportation | Numeric | Integer
pop2010 | total number of residents that reside in that county | Numeric | Integer
p_trans_acc | the percent of residents that live within 1/2 mile of public transportation | Numeric | Float
LL_95CI | lower limit of the 95th confidence interval for p_trans_acc | Numeric | Float
UL_95CI | upper limit of the 95th confidence interval for p_trans_acc | Numeric| Float
se | standard errror | Numeric| Float
rse | relative standard error | Numeric| Float

### Data Overview

In [33]:
# load tidied data and print rows
data = pd.read_csv(
    'tidy-data', 
    dtype = {'pop_trans_acc':'Int64',
             'county_fips': 'Int64'},
    index_col = 0)

data.head()

,year,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse
0,2008,3,AfricanAm,CO,6061,Placer,Placer,6061,Sacramento Area,8,55,4427,0.012424,0.009161,0.015687,0.001665,13.399974
1,2008,1,AIAN,CO,6061,Placer,Placer,6061,Sacramento Area,8,51,2080,0.024519,0.017873,0.031166,0.003391,13.830066
2,2008,2,Asian,CO,6061,Placer,Placer,6061,Sacramento Area,8,117,19963,0.005861,0.004802,0.006920,0.000540,9.217872
3,2008,4,Latino,CO,6061,Placer,Placer,6061,Sacramento Area,8,1835,44710,0.041042,0.039203,0.042881,0.000938,2.286029
4,2008,7,Multiple,CO,6061,Placer,Placer,6061,Sacramento Area,8,241,10658,0.022612,0.019790,0.025435,0.001440,6.368321


---
## 2. Initial explorations

At this stage, you may spend most of your effort on the computing side tidying up the data. You're not expected to complete a thorough exploratory analysis, and if your dataset was especially messy to start with, you may not even begin your exploratory analysis by the time you prepare this report. You have the option to leave exploration for the next stage of work and simply report basic properties of the dataset, but you should at minimum address the items in the 'basic properties' section below.

### Basic properties of the dataset

Help the reader get acquainted with your dataset on a simple level by identifying characteristics of the dataset and variable summaries. Some amount of code is fine here, but try to use code cells sparingly.

**Dimensions**: state the dimensions of the data (in tidy format, of course).

**Missing values**: Are there missing values? If so, why are they missing?

**Variable summaries**: Provide simple variable summaries for the most important variables in your dataset. Preferably, you'll do this for all variables, but if you have a large number, you might need to prioritize and focus on the ones most of interest. What exactly you do is a little case-specific, but think of things like means and variances, min/max, number of levels and observation counts for categorical variables, etc.

*Start your draft here.*

### Code:

**Dimensions**

In [34]:
data.shape

(66006, 17)

**Missing values**

In [35]:
missing_values = data.isna().sum()
missing_values

year                 0
race_eth_code        0
race_eth_name        0
geotype              0
geotypevalue         0
geoname             54
county_name         63
county_fips         63
region_name          0
region_code          0
pop_trans_acc     1182
pop2010              0
p_trans_acc       1557
LL_95CI           1182
UL_95CI           1557
se                1182
rse              14700
dtype: int64

**Variable summaries**

### Exploratory analysis

If you were lucky and your dataset was neat, you should aim to include a few exploratory plots or tables here -- they don't need to be polished at this stage, but you should select plots that are informative (rather than including all plots you may have looked at). 

If you do include exploratory graphics or tables, please explain in a sentence or two what each one shows. Try to include a minimum of code. Consider [saving your plots as images](https://altair-viz.github.io/user_guide/saving_charts.html#png-svg-and-pdf-format) and inputting images into markdown cells instead of generating them anew via code cells.

---
## 3. Planned work

Here you should indicate your tentative ideas for your analysis. Don't worry, these aren't final -- you can always change your mind later or shift gears if they don't pan out. The objective is to have you start thinking ahead about what you'll do.

### Questions

Please propose two focused questions that you plan to explore.

1. *Question 1 here*
2. *Question 2 here*

### Proposed approaches

For each question, please describe an idea or two about how you might approach the question.

1. *Approach 1 here* 
2. *Approach 2 here*

---
## Submission Checklist
1. Save file to confirm all changes are on disk
2. Run *Kernel > Restart & Run All* to execute all code from top to bottom
3. Save file again to write any new output to disk
4. Generate PDF and submit to Gradescope